# stock_reader デモ

pandas_datareaderライクなインターフェースでJ-Quantsの株価データにアクセスするデモです。

In [ ]:
# パッケージのインポートと初期化
from market_reader import DataReader

# DataReaderインスタンスを作成（デフォルトでjquants.dbを使用）
reader = DataReader()
print(f"Database: {reader.db_path}")

In [2]:
# 単一銘柄データ取得の例
# トヨタ(7203)の株価を取得
df_toyota = reader.get_prices("7203", start="2024-01-01", end="2024-12-31")

print(f"Shape: {df_toyota.shape}")
print(f"Index type: {type(df_toyota.index).__name__}")
print(f"Columns: {list(df_toyota.columns)}")
print()
df_toyota.head()

Shape: (245, 6)
Index type: DatetimeIndex
Columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentClose']



,Open,High,Low,Close,Volume,AdjustmentClose
Date,,,,,,
2024-01-04,2605.0,2635.0,2572.0,2635.0,29812900.0,2635.0
2024-01-05,2643.5,2708.0,2643.5,2701.5,30515500.0,2701.5
2024-01-09,2715.0,2716.0,2662.5,2694.0,30486100.0,2694.0
2024-01-10,2684.0,2752.0,2679.5,2745.0,33701200.0,2745.0
2024-01-11,2827.0,2867.5,2827.0,2844.0,49003100.0,2844.0


In [3]:
# 複数銘柄データ取得の例（MultiIndex操作）
# トヨタ(7203)とソフトバンク(9984)を同時取得
df_multi = reader.get_prices(
    ["7203", "9984"],
    start="2024-01-01",
    end="2024-01-31"
)

print(f"Shape: {df_multi.shape}")
print(f"Index type: {type(df_multi.index).__name__}")
print(f"Index names: {df_multi.index.names}")
print()

# MultiIndexへのアクセス例
print("=== 2024-01-04のデータ ===")
print(df_multi.loc["2024-01-04"])
print()

# 特定銘柄のみ抽出
print("=== トヨタのみ ===")
print(df_multi.xs("7203", level="Code").head())

Shape: (38, 6)
Index type: MultiIndex
Index names: ['Date', 'Code']

=== 2024-01-04のデータ ===
        Open    High     Low   Close      Volume  AdjustmentClose
Code                                                             
7203  2605.0  2635.0  2572.0  2635.0  29812900.0           2635.0
9984  6040.0  6086.0  6019.0  6050.0   9392500.0           6050.0

=== トヨタのみ ===
              Open    High     Low   Close      Volume  AdjustmentClose
Date                                                                   
2024-01-04  2605.0  2635.0  2572.0  2635.0  29812900.0           2635.0
2024-01-05  2643.5  2708.0  2643.5  2701.5  30515500.0           2701.5
2024-01-09  2715.0  2716.0  2662.5  2694.0  30486100.0           2694.0
2024-01-10  2684.0  2752.0  2679.5  2745.0  33701200.0           2745.0
2024-01-11  2827.0  2867.5  2827.0  2844.0  49003100.0           2844.0


In [4]:
# カラム選択の例

# simple: デフォルト（OHLCV + AdjustmentClose）
df_simple = reader.get_prices("7203", start="2024-01-01", end="2024-01-10", columns="simple")
print("=== columns='simple' ===")
print(list(df_simple.columns))
print()

# full: 全カラム
df_full = reader.get_prices("7203", start="2024-01-01", end="2024-01-10", columns="full")
print("=== columns='full' ===")
print(list(df_full.columns))
print()

# リスト指定
df_custom = reader.get_prices("7203", start="2024-01-01", end="2024-01-10", columns=["Open", "Close", "Volume"])
print("=== columns=['Open', 'Close', 'Volume'] ===")
print(list(df_custom.columns))

=== columns='simple' ===
['Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentClose']

=== columns='full' ===
['Open', 'High', 'Low', 'Close', 'UpperLimit', 'LowerLimit', 'Volume', 'TurnoverValue', 'AdjustmentFactor', 'AdjustmentOpen', 'AdjustmentHigh', 'AdjustmentLow', 'AdjustmentClose', 'AdjustmentVolume']

=== columns=['Open', 'Close', 'Volume'] ===
['Open', 'Close', 'Volume']


In [5]:
# 期間指定の例

# endのみ指定（startはendの1年前がデフォルト）
df_default_start = reader.get_prices("7203", end="2024-12-31")
print(f"Start date (default): {df_default_start.index.min()}")
print(f"End date: {df_default_start.index.max()}")
print()

# start/end両方省略（全期間）
df_all = reader.get_prices("7203")
print(f"Full range: {df_all.index.min()} to {df_all.index.max()}")
print(f"Total records: {len(df_all)}")

Start date (default): 2020-07-08 00:00:00
End date: 2024-12-30 00:00:00

Full range: 2021-01-29 00:00:00 to 2026-01-29 00:00:00
Total records: 1223


In [ ]:
# エラーハンドリングの例
from market_reader import StockNotFoundError
import warnings

# strict=False（デフォルト）: 存在しない銘柄は警告+空DataFrame
reader_lenient = DataReader(strict=False)
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    df_empty = reader_lenient.get_prices("9999", start="2024-01-01", end="2024-01-31")
    if w:
        print(f"Warning: {w[0].message}")
    print(f"Empty DataFrame: {df_empty.empty}")
print()

# strict=True: 存在しない銘柄は例外
reader_strict = DataReader(strict=True)
try:
    reader_strict.get_prices("9999", start="2024-01-01", end="2024-01-31")
except StockNotFoundError as e:
    print(f"StockNotFoundError: {e}")